# 🎨 Data Designer Tutorial: Seeding Synthetic Data Generation with an External Dataset

#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/1-the-basics/) in this tutorial series.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    ChatCompletionInferenceParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    ModelConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object is responsible for managing the data generation process.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) are used.


In [2]:
data_designer = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nemotron-3-nano-30b-a3b"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v3"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=ChatCompletionInferenceParams(
            temperature=1.0,
            top_p=1.0,
            max_tokens=2048,
            extra_body={"chat_template_kwargs": {"enable_thinking": False}},
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, _ = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

# Seed datasets are passed as reference objects to the config builder.
seed_dataset_reference = data_designer.make_seed_reference_from_file(local_filename)

config_builder.with_seed_dataset(seed_dataset_reference)

DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
)

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
Respond with only the notes, no other text.
""",
    model_alias=MODEL_ALIAS,
)

config_builder.validate()

[18:13:36] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
    sampler_columns: [
        "patient_sampler",
        "doctor_sampler",
        "patient_id",
        "symptom_onset_date",
        "date_of_visit"
    ]
    llm_text_columns: ['physician_notes']
    expression_columns: [
        "first_name",
        "last_name",
        "dob",
        "physician"
    ]
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer.preview(config_builder, num_records=2)

[18:13:36] [INFO] 👀 Preview generation in progress


[18:13:36] [INFO] ✅ Validation passed


[18:13:36] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[18:13:36] [INFO] 🩺 Running health checks for models...


[18:13:36] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v3'...


...

[18:13:42] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[18:13:42] [INFO]   |-- 🎲 column: 'date_of_visit'


[18:13:42] [INFO]   |-- 🧩 column: 'physician'


[18:13:42] [INFO]   |-- 📝 column: 'physician_notes'


[18:13:42] [INFO] 🥳 Preview complete!


In [8]:
# Run this cell multiple times to cycle through the 2 preview records.
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis       │ cervical spondylosis                                                                          │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ patient_summary │ I've been having a lot of pain in my neck and back. I've also been having trouble with my     │
│                 │ balance and coordination. I've been coughing a lot and my limbs feel weak.                    │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': 'a40f61e8-4dc1-4cd5-9e9d-dc2b41dae16b',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'Jason',                                                                 │
│                    │     'last_name': 'Herrera',                                                                │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Male',                                                                         │
│                    │     'street_number': '0770',                                                               │
│                    │     'street_name': 'Joseph Bypass',                                                        │
│                    │     'city': 'Joshuahaven',                                                                 │
│                    │     'state': 'California',                                                                 │
│                    │     'postcode': '17152',                                                                   │
│                    │     'age': 49,                                                                             │
│                    │     'birth_date': '1976-09-30',                                                            │
│                    │     'country': 'Tuvalu',                                                                   │
│                    │     'marital_status': 'widowed',                                                           │
│                    │     'education_level': 'bachelors',                                                        │
│                    │     'unit': '',                                                                            │
│                    │     'occupation': 'Bonds trader',                                                          │
│                    │     'phone_number': '789.327.9784

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': 'a40f61e8-4dc1-4cd5-9e9d-dc2b41dae16b...,{'uuid': 'e445a566-86ae-4c20-a738-af62d03b1c7a...,PT-33818DAF,2024-07-19,2024-07-29,Jason,Herrera,1976-09-30,Dr. Butler,DR SOS PAT 7/29/24 14:30\nPT: Jason Herrera 35...
1,impetigo,I have a rash on my face that is getting worse...,{'uuid': 'fcc8230e-1fb8-44df-9017-9f584846cc86...,{'uuid': 'd211f307-95d0-4dc5-99a9-03be82cf8b72...,PT-B50E0245,2024-03-14,2024-04-04,Peter,Perry,1998-03-22,Dr. Payne,**Patient:** Peter Perry **DOB:** 1988-06-12 *...


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2                               │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                     2 (100.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                     2 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                       2 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                       2 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                       2 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                       2 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                       2 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
results = data_designer.create(config_builder, num_records=10, dataset_name="tutorial-3")

[18:13:42] [INFO] 🎨 Creating Data Designer dataset


[18:13:42] [INFO] ✅ Validation passed


[18:13:42] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[18:13:42] [INFO] 🩺 Running health checks for models...


[18:13:42] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v3'...


...

[18:13:56] [INFO]   |-- 🧩 column: 'dob'


[18:13:56] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[18:13:56] [INFO]   |-- 🎲 column: 'date_of_visit'


[18:13:56] [INFO]   |-- 🧩 column: 'physician'


[18:13:56] [INFO]   |-- 📝 column: 'physician_notes'


In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 80, 'bachelors_field': 'business', 'bi...","{'age': 23, 'bachelors_field': 'no_degree', 'b...",PT-3FD09135,2024-08-09,2024-08-21,Deborah,Sosa,1945-04-19,Dr. Summers,S: 63 y/o F w/ hx cervical spondylosis (dx 202...
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 107, 'bachelors_field': 'stem', 'birth...","{'age': 34, 'bachelors_field': 'stem', 'birth_...",PT-1639D137,2024-06-07,2024-07-05,Marie,Solis,1918-10-09,Dr. Sullivan,**Patient:** Marie Solis **DOB:** 02/15/1990...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 113, 'bachelors_field': 'no_degree', '...","{'age': 94, 'bachelors_field': 'no_degree', 'b...",PT-52EE8B67,2024-06-12,2024-07-02,Jonathan,Turner,1912-10-07,Dr. Williams,*** URGENT CARE NOTES - DR. C. WILLIAMS *** ...
3,arthritis,I have been having trouble with my muscles and...,"{'age': 102, 'bachelors_field': 'no_degree', '...","{'age': 98, 'bachelors_field': 'no_degree', 'b...",PT-23BDC3BC,2024-05-10,2024-06-02,Brian,Gardner,1922-12-21,Dr. Ramirez,"**Visit Summary – Dr. Gregory Ramirez, MD** ..."
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 48, 'bachelors_field': 'business', 'bi...","{'age': 76, 'bachelors_field': 'business', 'bi...",PT-B0E70E70,2024-02-25,2024-03-14,Sherry,Hayes,1977-05-10,Dr. Freeman,"**SOAP Note – Dr. Jill Freeman, MD** **Date:..."


In [13]:
# Load the analysis results into memory.
analysis = results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                      7 (70.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    10 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      10 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      10 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                      10 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

## ⏭️ Next Steps

Check out the following notebook to learn more about:

- [Providing images as context](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/4-providing-images-as-context/)
